<h1><center>Finding Similar Songs on Spotify<br><br>Part 1: Nearest Neighbor Search</center></h1>

The first part of this tutorial series demonstrates the traditional way of extracting features from the audio content, training a classifier and predicting results. Because we do not have access to the raw audio content, we cannot extract features ourselves. Fortunately, Spotify is so generious to provide extracted features via their API. Those are just low-level audio features, but they are more than any other streaming music service provide - so Kudos to Spotify for this API! 


## Part 1 - Overview

1. Accessing the Spotify API
2. Loading data
3. Preprocess data
4. Define the Similarity Model
4. Optimize the Model
5. Evaluate the Models



## Requiremnts

Install the following dependencies to run this tutorial:

In [710]:
# visualization
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

# numeric and scientific processing
import numpy as np
import pandas as pd

# misc
import os
import progressbar

**Spotipy**

Spotipy is a thin client library for the Spotify Web API.

https://github.com/plamere/spotipy

In [711]:
import spotipy
import spotipy.util as util

TODO: describe how to get client credentials

In [712]:
os.environ["SPOTIPY_CLIENT_ID"]     = "8a7fffc37b6c44e6b7bc344c3295034c"
os.environ["SPOTIPY_CLIENT_SECRET"] = "f19dd914ba58408c9407dd6479b23812"

If you get the following message:

    User authentication requires interaction with your
    web browser. Once you enter your credentials and
    give authorization, you will be redirected to
    a url.  Paste that url you were directed to to
    complete the authorization.

    Opened https://accounts.spotify.com/authorize?scope=playlist-modify-public&redirect_uri=ht...
    

You need to authenticate your browser session. Follow the link and log in to Spotify. After login, you will be redirected to http://localhost/?code=... Copy the entire URL and paste it to the prompted textbox.

In [713]:
token = util.prompt_for_user_token("slychief", 
                                   "playlist-modify-public", 
                                   redirect_uri="http://localhost/")

sp = spotipy.Spotify(auth=token)

# Loading Data

Before we can train our models we first have to get some data.

## Download Echonest Features from Spotify

We use spotipy to access the Spotify API and to download metadata and audio features of Spotify tracks. The following list provides a selection of Spotify playlists of various music genres.

In [714]:
playlists = [
    
     {"name": "clubbeats",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DXbX3zSzB4MO0"},
     {"name": "softpop",      "uri": "spotify:user:spotify:playlist:37i9dQZF1DWTwnEm1IYyoj"},
     {"name": "electropop",   "uri": "spotify:user:spotify:playlist:37i9dQZF1DX4uPi2roRUwU"},
     {"name": "rockclassics", "uri": "spotify:user:spotify:playlist:37i9dQZF1DWXRqgorJj26U"},
     {"name": "rockhymns",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DX4vth7idTQch"},
     {"name": "soft_rock",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DX6xOPeSOGone"},
     {"name": "metalcore",    "uri": "spotify:user:spotify:playlist:37i9dQZF1DWXIcbzpLauPS"}, 
     {"name": "metal",        "uri": "spotify:user:spotify:playlist:37i9dQZF1DWWOaP4H0w5b0"},
     {"name": "classic_metal","uri": "spotify:user:spotify:playlist:37i9dQZF1DX2LTcinqsO68"},
     {"name": "grunge",       "uri": "spotify:user:spotify:playlist:37i9dQZF1DX11ghcIxjcjE"},
     {"name": "hiphop",       "uri": "spotify:user:spotify:playlist:37i9dQZF1DWVdgXTbYm2r0"},
     {"name": "poppunk",      "uri": "spotify:user:spotify:playlist:37i9dQZF1DXa9wYJr1oMFq"},
     {"name": "classic",      "uri": "spotify:user:spotify:playlist:37i9dQZF1DXcN1fAVSf7CR"}
    
]

### Get Playlist meta-data

Insted of writing one big loop to download the data, I decided to split it into separate more comprehensible steps.

The Spotify API does not return infinite elements, but requires batch processing. The largest batch size is 100 items such as tracks, artists or albums. As a first step we get relevant meta-data for the supplied playlists. Especially the *num_track* property is conveniant for the further processing.

In [715]:
def get_playlist_metadata(spotify_client, playlists):

    for playlist in playlists:

        # get user and playlist_id from uri
        (_,_,user,_,playlist_id) = playlist["uri"].split(":")

        # retrieve playlist metadat from Spotify
        playlist_metadata = spotify_client.user_playlist(user        = user,
                                                         playlist_id = playlist_id)

        # extract relevant information
        playlist["user"]        = user
        playlist["playlist_id"] = playlist_id
        playlist["num_tracks"]  = playlist_metadata["tracks"]["total"]

        # initialize fields for further processing
        playlist["track_ids"]   = []
        
    return playlists

In [716]:
playlists = get_playlist_metadata(sp, playlists)

**Size of our dataset**

Now we can already estimate the approximate size of our data-set. The set might contain duplicate tracks from overlapping playlists. Those are removed later on.

In [717]:
num_tracks_total = np.sum([playlist["num_tracks"] for playlist in playlists])
num_tracks_total

970

### Get track-ids of all playlist entries

To download meta- and feature-data for tracks, we need to fetch the playlist entries.

In [718]:
def get_track_ids(sp, playlists):

    # max Spotify batch size
    batch_size = 100

    # retrieve tracks for each playlist
    for playlist in playlists:

        # batch processing
        for offset in np.arange(0, playlist["num_tracks"], batch_size):

            limit = np.min([batch_size, playlist["num_tracks"] - offset])

            playlist_entries = sp.user_playlist_tracks(user        = playlist["user"],
                                                       playlist_id = playlist["playlist_id"], 
                                                       limit       = limit, 
                                                       offset      = offset,
                                                       fields      = ["items"])

            playlist["track_ids"].extend([entry["track"]["id"] for entry in playlist_entries["items"]])
            
    return playlists

In [719]:
playlists = get_track_ids(sp, playlists)

### Fetch data and features from Spotify

Now it's time to download the dat for our tracks

#### Caching with joblib

We will use caching to locally store retrieved data. This is on the one hand a requirement of the API and on the other it speeds up processing when we reload the notebook. *joblib* is a convenient library which simplifies caching.

*Update the cachdir to an appropriate path in the following cell*

In [82]:
from joblib import Memory

memory = Memory(cachedir='/home/schindler/tmp/spotify/', verbose=0)

The following method retrieves meta-data, sequential features such as *MFCCs* and *Chroma*, and track-level features such as *Dancability*. The *@memory.cache* annotation tells *joblib* to persist all return values for the supplied parameters.

In [83]:
@memory.cache
def get_spotify_data(track_id):
    
    # meta-data
    track_metadata      = sp.track(track_id)
    album_metadata      = sp.album(track_metadata["album"]["id"])
    artist_metadata     = sp.artist(track_metadata["artists"][0]["id"])
    
    # feature-data
    sequential_features = sp.audio_analysis(track_id)
    trackbased_features = sp.audio_features([track_id])
    
    return track_metadata, album_metadata, artist_metadata, sequential_features, trackbased_features

**Retrieving data for all tracks**

The following loop downloads meta- and feature-data for all tracks. The *processed_track_ids* list is used to avoid duplicated entries. Be aware that downloading a lot of tracks could take some time. Processing 1.000 tracks may take about 15 to 30 minutes.

In [720]:
def download_track_data(sp, playlists):

    pbar = progressbar.ProgressBar(max_value=num_tracks_total)
    pbar.start()

    raw_track_data      = []
    processed_track_ids = []

    for playlist in playlists:

        for track_id in playlist["track_ids"]:

            try:
                # avoid duplicates in the data-set
                if track_id not in processed_track_ids:

                    # retrieve data from Spotify
                    spotify_data = get_spotify_data(track_id)

                    raw_track_data.append([playlist["name"], spotify_data])
                    processed_track_ids.append(track_id)

            except Exception as e:
                print e

            pbar.update(len(raw_track_data))
            
    return raw_track_data

In [721]:
raw_track_data = download_track_data(sp, playlists)

 62% (606 of 970) |#################################################################################################################                                                                     | Elapsed Time: 0:00:25 ETA: 0:00:20

Unterminated string starting at: line 1 column 65530 (char 65529)


 97% (942 of 970) |################################################################################################################################################################################      | Elapsed Time: 0:00:40 ETA: 0:00:01

# Aggregate data

Currently we only have a list of raw data-objects retrieved from the Spotify API. We need to transform this information to a more structured format.

## Aggregate Meta-data

First we aggregate the meta-data. All relevant information is stored in a dataframe.

In [722]:
def aggregate_metadata(raw_track_data):

    metadata = []

    for playlist_name, spotify_data in raw_track_data:

        track_metadata, album_metadata, artist_metadata, _, _ = spotify_data

        # get year of album release
        release_date = album_metadata["release_date"]

        if album_metadata["release_date_precision"] != "year":
            release_date = release_date.split("-")[0]

        # assamble metadata
        metadata.append([track_metadata["id"],
                         artist_metadata["name"], 
                         track_metadata["name"], 
                         album_metadata["name"],
                         album_metadata["label"],
                         track_metadata["duration_ms"],
                         track_metadata["popularity"],
                         release_date,
                         artist_metadata["genres"], 
                         playlist_name])

    metadata = pd.DataFrame(metadata, columns=["track_id", "artist_name", "title", "album_name", "label", 
                                               "duration", "popularity",  "year",  "genres", "playlist"])
    
    return metadata

In [723]:
metadata = aggregate_metadata(raw_track_data)

Here an example of the aggregated meta-data:

In [724]:
metadata.sort_values("popularity", ascending=False).head()

,track_id,artist_name,title,album_name,label,duration,popularity,year,genres,playlist
800,2eMwDehkIC1j68U6FA3Eiq,Lil Uzi Vert,XO TOUR Llif3,XO TOUR Llif3,Generation Now/Atlantic,182706,96,2017,"[dwn trap, rap]",hiphop
805,5eqK0tbzUPo2SoeZsov04s,21 Savage,Bank Account,Issa Album,"Slaughter Gang, LLC/Epic Records",220306,96,2017,"[dwn trap, rap, trap music]",hiphop
100,7qiZfU4dY1lWllzX7mPBI3,Ed Sheeran,Shape of You,÷ (Deluxe),Atlantic Records UK,233712,96,2017,[pop],softpop
96,1wjzFQodRWrPcQ0AnYnvQ9,Lauv,I Like Me Better,I Like Me Better,Lauv,197436,96,2017,"[indie poptimism, pop, post-teen pop, tropical...",softpop
248,4q8PHoRsPUB52LFylX8Ulz,Axwell /\ Ingrosso,More Than You Know,More Than You Know,Digital Distribution Trinidad and Tobago,203000,96,2017,"[big room, edm, electro house, pop, tropical h...",electropop


## Aggregate Features



In [694]:
def aggregate_features(seq_data, track_data, metadata, with_year=False, with_popularity=False):

    calc_statistical_moments = lambda x: np.concatenate([x.mean(axis=0), x.std(axis=0)])
    
    # sequential data
    segments = seq_data["segments"]
    sl       = len(segments)
    
    # MFCCs - 24 dimensions
    mfcc              = np.array([s["timbre"] for s in segments])
    mfcc              = calc_statistical_moments(mfcc)
    
    # Chroma / pitch classes - 24 dimensions
    chroma            = np.array([s["pitches"] for s in segments])
    chroma            = calc_statistical_moments(chroma)
    
    # maximum loudness values per segment - 2 dimensions
    loudness_max      = np.array([s["loudness_max"] for s in segments]).reshape((sl,1))
    loudness_max      = calc_statistical_moments(loudness_max)
    
    # offset of max loudness value within segment - 2 dimensions
    loudness_start    = np.array([s["loudness_start"] for s in segments]).reshape((sl,1))
    loudness_start    = calc_statistical_moments(loudness_start)
    
    # length of max loudness values within segment - 2 dimensions
    loudness_max_time = np.array([s["loudness_max_time"] for s in segments]).reshape((sl,1))
    loudness_max_time = calc_statistical_moments(loudness_max_time)
    
    # length of segment - 2 dimensions
    duration          = np.array([s["duration"] for s in segments]).reshape((sl,1))
    duration          = calc_statistical_moments(duration)
    
    # confidence of segment boundary detection - 2 dimensions
    confidence        = np.array([s["confidence"] for s in segments]).reshape((sl,1))
    confidence        = calc_statistical_moments(confidence)
    
    # concatenate sequential features
    sequential_features = np.concatenate([mfcc, chroma, loudness_max, loudness_start, 
                                          loudness_max_time, duration, confidence], axis=0)
    
    # track-based data
    track_features = [track_data[0]["acousticness"],     # acoustic or not?
                      track_data[0]["danceability"],     # danceable?
                      track_data[0]["energy"],           # energetic or calm?
                      track_data[0]["instrumentalness"], # is somebody singing?
                      track_data[0]["liveness"],         # live or studio?
                      track_data[0]["speechiness"],      # rap or singing?
                      track_data[0]["tempo"],            # slow or fast?
                      track_data[0]["time_signature"],   # 3/4, 4/4, 6/8, etc.
                      track_data[0]["valence"]]          # happy or sad?
    
    if with_year:
        track_features.append(int(metadata["year"]))
        
    if with_popularity:
        track_features.append(int(metadata["popularity"]))
        
    
    return np.concatenate([sequential_features, track_features], axis=0)

Afgregate all features of the downloaded data

In [725]:
def aggregate_featuredata(raw_track_data):

    feature_data = []

    for i, (_, spotify_data) in enumerate(raw_track_data):

        _, _, _, f_sequential, f_trackbased = spotify_data

        feature_vec = aggregate_features(f_sequential, 
                                         f_trackbased, 
                                         metadata.iloc[i], 
                                         with_year       = True, 
                                         with_popularity = True)    

        feature_data.append(feature_vec)

    feature_data = np.asarray(feature_data)
    
    return feature_data

In [726]:
feature_data = aggregate_featuredata(raw_track_data)

print "feature_data.shape:", feature_data.shape

feature_data.shape: (944, 69)


### Normalize feature data

In [696]:
# standardize sequential_features
feature_data -= feature_data.mean(axis=0)
feature_data /= feature_data.std(axis=0)

# Calculate Similarities

### Euclidean Distance

In [698]:
eucledian_distance = lambda feature_space, query_vector: \
                                np.sqrt(np.sum((feature_space - query_vector)**2, axis=1))

In [700]:
query_track_idx = 381

In [702]:
# calculate the distance between the query-vector and all others
dist = eucledian_distance(feature_data, feature_data[query_track_idx])

# sort the distances ascendingly - use sorted index
sorted_idx = np.argsort(dist)

# display top-10 results (first track = query track)
display_cols = ["artist_name", "title", "album_name", "year", "playlist"]

metadata.loc[sorted_idx[:11], display_cols]

,artist_name,title,album_name,year,playlist
381,Lynyrd Skynyrd,Sweet Home Alabama,Second Helping,1974,rockclassics
445,Green Day,Good Riddance (Time Of Your Life),Nimrod,1997,soft_rock
324,Cream,White Room,Wheels Of Fire (Remastered),1968,rockclassics
450,New Radicals,You Get What You Give,Maybe You've Been Brainwashed Too,1998,soft_rock
750,Candlebox,Far Behind,Candlebox,1993,grunge
473,America,Ventura Highway,Homecoming,1972,soft_rock
389,Steve Miller Band,Take The Money And Run,Fly Like An Eagle,1976,rockclassics
458,Heart,All I Wanna Do Is Make Love To You,Brigade,1990,soft_rock
448,Bruce Hornsby,The Way It Is,The Way It Is,1986,soft_rock
300,Bob Seger,Still The Same,Stranger In Town,1978,rockclassics


### Scaled Eucledian Distance

prblem: large featuresets dominate

In [555]:
# feature-set lengths and order
featureset_lengths = [24, # mfcc
                      24, # chroma
                      2,  # loudness_max
                      2,  # loudness_start
                      2,  # loudness_max_time
                      2,  # sequence length
                      2,  # confidence
                      1,  # acousticness
                      1,  # danceability
                      1,  # energy
                      1,  # instrumentalness
                      1,  # liveness
                      1,  # speechiness
                      1,  # tempo
                      1,  # time_signature
                      1,  # valence
                      1,  # year
                      1]  # popularity

In [556]:
def scaled_eucledian_distance(feature_space, query_vector):
    
    distances = (feature_space - query_vector)**2
    
    # feature_start_idx
    start_idx = 0 
    
    # normalize distances
    for sequence_length in featureset_lengths:
        
        # feature_stop_idx
        stop_idx                         = start_idx + sequence_length
        distances[:,start_idx:stop_idx] /= distances[:,start_idx:stop_idx].sum(axis=1).max()
        start_idx                        = stop_idx
    
    return np.sqrt(np.sum(distances, axis=1))

In [704]:
dist = scaled_eucledian_distance(feature_data, feature_data[query_track_idx])

metadata.loc[np.argsort(dist)[:11], display_cols]

,artist_name,title,album_name,year,playlist
381,Lynyrd Skynyrd,Sweet Home Alabama,Second Helping,1974,rockclassics
389,Steve Miller Band,Take The Money And Run,Fly Like An Eagle,1976,rockclassics
473,America,Ventura Highway,Homecoming,1972,soft_rock
432,Deep Purple,Smoke On the Water,Machine Head (Remastered),1972,rockhymns
458,Heart,All I Wanna Do Is Make Love To You,Brigade,1990,soft_rock
475,Crowded House,Weather With You,Woodface,1991,soft_rock
324,Cream,White Room,Wheels Of Fire (Remastered),1968,rockclassics
277,Eagles,Life In The Fast Lane,Hotel California (Remastered),1976,rockclassics
354,Aerosmith,Walk This Way,Aerosmith's Greatest Hits,1980,rockclassics
393,The Doobie Brothers,Listen To The Music,Toulouse Street,1972,rockclassics


### Feature Weighting

In [675]:
# feature-set lengths and order
featureset_weights = [1.55,  # mfcc
                      0.6,  # chroma
                      0.8,  # loudness_max
                      0.5,  # loudness_start
                      0.5,  # loudness_max_time
                      2.5,  # sequence length
                      0.5,  # confidence
                      0.5,  # acousticness
                      0.5,  # danceability
                      1.5,  # energy
                      0.0,  # instrumentalness
                      0.0,  # liveness
                      0.1,  # speechiness
                      0.1,  # tempo
                      0.9,  # time_signature
                      0.1,  # valence
                      5.5,  # year
                      0.4]  # popularity

In [676]:
def weighted_eucledian_distance(feature_space, query_vector, featureset_weights):
    
    distances = (feature_space - query_vector)**2
    
    # feature_start_idx
    start_idx = 0 
    
    # normalize distances
    for sequence_length, weight in zip(featureset_lengths, featureset_weights):

        # feature_stop_idx
        stop_idx                         = start_idx + sequence_length
        distances[:,start_idx:stop_idx] /= distances[:,start_idx:stop_idx].sum(axis=1).max()
        distances[:,start_idx:stop_idx] *= weight
        start_idx                        = stop_idx

    return np.sqrt(np.sum(distances, axis=1))

In [705]:
dist = weighted_eucledian_distance(feature_data, feature_data[query_track_idx], featureset_weights)

metadata.loc[np.argsort(dist)[:11], display_cols]

,artist_name,title,album_name,year,playlist
381,Lynyrd Skynyrd,Sweet Home Alabama,Second Helping,1974,rockclassics
306,Bad Company,Feel Like Makin' Love - 2015 Remastered Version,Straight Shooter,1975,rockclassics
389,Steve Miller Band,Take The Money And Run,Fly Like An Eagle,1976,rockclassics
473,America,Ventura Highway,Homecoming,1972,soft_rock
324,Cream,White Room,Wheels Of Fire (Remastered),1968,rockclassics
432,Deep Purple,Smoke On the Water,Machine Head (Remastered),1972,rockhymns
393,The Doobie Brothers,Listen To The Music,Toulouse Street,1972,rockclassics
295,Eagles,Hotel California,Hotel California (Remastered),1976,rockclassics
277,Eagles,Life In The Fast Lane,Hotel California (Remastered),1976,rockclassics
338,The Doors,Love Her Madly,L.A. Woman,1971,rockclassics


# Evaluate

In [565]:
def evaluate(similarity_function, cut_off):

    all_precisions = []

    for idx in metadata.index.values:

        dist           = similarity_function(feature_data, feature_data[idx])
        similar_tracks = metadata.loc[np.argsort(dist)[:cut_off]]
        same_label     = similar_tracks["playlist"] == metadata.loc[idx, "playlist"]
        precision      = same_label.sum() / float(cut_off)
        all_precisions.append(precision)

    all_precisions = np.array(all_precisions)

    return all_precisions.mean()

In [685]:
evaluate(lambda x,y: weighted_eucledian_distance(x,y, featureset_weights), 20)

0.58159574468085107

In [686]:
evaluate(lambda x,y: scaled_eucledian_distance(x,y), 20)

0.49632978723404253

In [687]:
evaluate(lambda x,y: eucledian_distance(x,y), 20)

0.43276595744680851